In [1]:
import numpy as np
import pandas as pd
import timeit

from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

from pyspark.ml.feature import Binarizer
from pyspark.ml.feature import VectorAssembler  # OneHotEncoder, VectorAssembler, StringIndexer, VectorIndexer
from pyspark.ml.classification import LogisticRegression

In [2]:
spark = SparkSession(SparkContext('local'))

21/12/29 22:19:35 WARN Utils: Your hostname, nour-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
21/12/29 22:19:36 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/29 22:19:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
def data_with_notint_col(dataset,output_col):
    not_int_col = [c for c in dataset.columns if dataset[c].dtype != 'int64' and c!=output_col]  # to get coloumn with object type
    [(c,dataset[c].unique()) for c in not_int_col]  # get string column with itis unique values
    for c in not_int_col:
        # 'c' -> column_name, change to value it's index  # it will be replaced with 0, 1, .....
        dataset[c] = dataset[c].apply({value:index for index, value in enumerate(list(dataset[c].unique()))}.get)  # to change data from object to int
    return dataset

In [4]:
file = pd.read_csv('test/adult.csv',delimiter=", ")

/home/nour/.local/lib/python3.8/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [5]:
file.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [6]:
data = data_with_notint_col(file,'')

In [7]:
data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,0,77516,0,13,0,0,0,0,0,2174,0,40,0,0
1,50,1,83311,0,13,1,1,1,0,0,0,0,13,0,0
2,38,2,215646,1,9,2,2,0,0,0,0,0,40,0,0
3,53,2,234721,2,7,1,2,1,1,0,0,0,40,0,0
4,28,2,338409,0,13,1,3,2,1,1,0,0,40,1,0


In [8]:
data = data.drop(columns=['fnlwgt'])

In [9]:
data.head()

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,0,0,13,0,0,0,0,0,2174,0,40,0,0
1,50,1,0,13,1,1,1,0,0,0,0,13,0,0
2,38,2,1,9,2,2,0,0,0,0,0,40,0,0
3,53,2,2,7,1,2,1,1,0,0,0,40,0,0
4,28,2,0,13,1,3,2,1,1,0,0,40,1,0


In [10]:
data = pd.concat([data]*20, ignore_index=True) # Ignores the index

In [11]:
data.to_csv("Data.csv",index=False)

In [12]:
df = spark.read.format('com.databricks.spark.csv').option("delimiter", ",").options(header='true', inferschema='true').load('Data.csv')
df.columns

['age',
 'workclass',
 'education',
 'education-num',
 'marital-status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'capital-gain',
 'capital-loss',
 'hours-per-week',
 'native-country',
 'salary']

In [13]:
print(df.count(), len(df.columns))

651220 14


In [14]:
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- workclass: integer (nullable = true)
 |-- education: integer (nullable = true)
 |-- education-num: integer (nullable = true)
 |-- marital-status: integer (nullable = true)
 |-- occupation: integer (nullable = true)
 |-- relationship: integer (nullable = true)
 |-- race: integer (nullable = true)
 |-- sex: integer (nullable = true)
 |-- capital-gain: integer (nullable = true)
 |-- capital-loss: integer (nullable = true)
 |-- hours-per-week: integer (nullable = true)
 |-- native-country: integer (nullable = true)
 |-- salary: integer (nullable = true)



In [15]:
df.groupBy('sex').count().show()

+---+------+
|sex| count|
+---+------+
|  1|215420|
|  0|435800|
+---+------+



In [16]:
featureColumns = ['age',
 'workclass',
 'education',
 'education-num',
 'marital-status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'capital-gain',
 'capital-loss',
 'hours-per-week',
 'native-country']

In [17]:
for col in df.columns:
     df = df.withColumn(col, df[col].cast("double"))

In [18]:
binarizer = Binarizer(threshold=0.0, inputCol='salary', outputCol="label")
binarizedDF = binarizer.transform(df)
binarizedDF = binarizedDF.drop('salary')

In [19]:
assembler = VectorAssembler(inputCols = featureColumns, outputCol = "features")
assembled = assembler.transform(binarizedDF)
print(assembled)

DataFrame[age: double, workclass: double, education: double, education-num: double, marital-status: double, occupation: double, relationship: double, race: double, sex: double, capital-gain: double, capital-loss: double, hours-per-week: double, native-country: double, label: double, features: vector]


In [20]:
(trainingData, testData) = assembled.randomSplit([0.7, 0.3], seed=0)
print("Distribution of Ones and Zeros in trainingData is: ", trainingData.groupBy("label").count().take(3))

21/12/29 22:20:08 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Distribution of Ones and Zeros in trainingData is:  [Row(label=0.0, count=345681), Row(label=1.0, count=109626)]


In [21]:
start = timeit.default_timer()
LR = LogisticRegression(maxIter=10000, regParam=0.3, elasticNetParam=0.8)

# Fit the model
LRModel = LR.fit(trainingData)


In [22]:
trainingSummary = LRModel.summary

In [23]:
accuracy = trainingSummary.accuracy
falsePositiveRate = trainingSummary.weightedFalsePositiveRate
truePositiveRate = trainingSummary.weightedTruePositiveRate
fMeasure = trainingSummary.weightedFMeasure()
precision = trainingSummary.weightedPrecision
recall = trainingSummary.weightedRecall
print("Accuracy: %s\nFPR: %s\nTPR: %s\nF-measure: %s\nPrecision: %s\nRecall: %s"
      % (accuracy, falsePositiveRate, truePositiveRate, fMeasure, precision, recall))

Accuracy: 0.7592261924371907
FPR: 0.7592261924371907
TPR: 0.7592261924371907
F-measure: 0.655315858484473
Precision: 0.5764244112826742
Recall: 0.7592261924371907


In [24]:
stop = timeit.default_timer()

print('Time: ', round(stop - start,2),"S")

Time:  21.31 S


In [25]:
spark